### grid

In [ ]:
from sklearn.datasets import load_digits
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

if __name__ == "__main__":
    X, y = load_digits(n_class=10, return_X_y=True)
    clf = LogisticRegression()

    param_grid = {'C': np.arange(0, 10),
                  "penalty": ['l1', 'l2'],
                  "class_weight": ["balanced", None]
                  }

    model = GridSearchCV(
        estimator=clf,
        param_grid=param_grid,
        cv=10,
        verbose=10,
        n_jobs=1,
        scoring="accuracy"
    )

    model.fit(X, y)
    print(f"Best score: {model.best_score_}")

    print("Best parameters set:")
    best_parameters = model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
        print(f"\t{param_name}: {best_parameters[param_name]}")

### random

In [ ]:
from sklearn.datasets import load_digits
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

if __name__ == "__main__":
    X, y = load_digits(n_class=10, return_X_y=True)
    clf = LogisticRegression()

    param_grid = {'C': np.arange(0, 10),
                  "penalty": ['l1', 'l2'],
                  "class_weight": ["balanced", None]
                  }

    model = RandomizedSearchCV(
        estimator=clf,
        param_distributions=param_grid,
        cv=10,
        verbose=10,
        n_jobs=1,
        scoring="accuracy",
        n_iter= 10
    )

    model.fit(X, y)
    print(f"Best score: {model.best_score_}")

    print("Best parameters set:")
    best_parameters = model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
        print(f"\t{param_name}: {best_parameters[param_name]}")

## hyopt

In [ ]:
import numpy as np
import pandas as pd
from functools import partial
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.datasets import load_digits


def optimize(params, x, y):
    
    model = ensemble.RandomForestClassifier(**params)
   
    kf = model_selection.StratifiedKFold(n_splits=5)

    accuracies = []

    for idx in kf.split(X=x, y=y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]
        xtest = x[test_idx]
        ytest = y[test_idx]
        
        model.fit(xtrain, ytrain)
    
        preds = model.predict(xtest)
        
        fold_accuracy = metrics.accuracy_score(
            ytest,
            preds
        )
        accuracies.append(fold_accuracy)

    return -1 * np.mean(accuracies)


if __name__ == "__main__":
    x, y = load_digits(n_class=10, return_X_y=True)

    param_space = {
        "max_depth": scope.int(hp.quniform("max_depth", 1, 15, 1)),
        "n_estimators": scope.int(hp.quniform("n_estimators", 100, 1500, 1)),
        "criterion": hp.choice("criterion", ["gini", "entropy"]),
        "max_features": hp.uniform("max_features", 0, 1)
    }
  
    optimization_function = partial(optimize,x=x,y=y)
    trials = Trials()

    hopt = fmin(
        fn=optimization_function,
        space=param_space,
        algo=tpe.suggest,
        max_evals=15,
        trials=trials
    )

    print(hopt)

### skopt

In [ ]:
import numpy as np
import pandas as pd
from functools import partial
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from skopt import gp_minimize,space
from sklearn.datasets import load_digits


def optimize(params, param_names, x, y):

    params = dict(zip(param_names, params))

    model = ensemble.RandomForestClassifier(**params)

    kf = model_selection.StratifiedKFold(n_splits=5)

    accuracies = []

    for idx in kf.split(X=x, y=y):
        train_idx, test_idx = idx[0], idx[1]
        xtrain = x[train_idx]
        ytrain = y[train_idx]
        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        preds = model.predict(xtest)

        fold_accuracy = metrics.accuracy_score(
            ytest,
            preds
        )
        accuracies.append(fold_accuracy)

    return -1 * np.mean(accuracies)


if __name__ == "__main__":
    x, y = load_digits(n_class=10, return_X_y=True)

    
    param_space = [
        space.Integer(3, 15, name="max_depth"),
        space.Integer(100, 1500, name="n_estimators"),
        space.Categorical(["gini", "entropy"], name="criterion"),
        space.Real(0.01, 1, prior="uniform", name="max_features")
    ]

    param_names = [
        "max_depth",
        "n_estimators",
        "criterion",
        "max_features"
    ]

    optimization_function = partial(
        optimize,
        param_names=param_names,
        x=x,
        y=y
    )


    result = gp_minimize(
        optimization_function,
        dimensions=param_space,
        n_calls=15,
        n_random_starts=10,
        verbose=10
    )

    best_params = dict(
        zip(
            param_names,
            result.x
        )
    )
    print(best_params)